In [53]:
def chkpossiblepalindrome(s):
    g = []
    n = len(s)
    leftHalf = ''
    mid = ''
    if(n%2 == 1):
        OddChar=''    
    for i in range(n):
        currChar=s[i]
        if currChar not in g:
            currCharRpt = s.count(currChar)
            g.append(currChar)
            if(n > 0 and n%2 == 0 and currCharRpt % 2 == 1):
                return (False,'')
            elif(n%2 == 1 and currCharRpt % 2 == 1 and OddChar != ''):
                return (False,'')
            elif(n%2 == 1 and currCharRpt % 2 == 1 and OddChar == ''):
                OddChar = currChar
                mid = OddChar * currCharRpt
            elif(n > 0 and currCharRpt % 2 == 0):
                leftHalf += currChar*int(currCharRpt/2)
    return (True,leftHalf + mid + leftHalf[::-1])

def gcd(a,b):
    if a < 0:
        a = -a
    if b < 0:
        b = -b
    if(b == 0):
        return a
    else:
        return gcd(b,a%b)
def gcdlist(l):
    if(len(l) == 1):
        return l[0]
    else:
        return gcd(l[0],gcdlist(l[1:]))


def fastmodexp(c, x, m):
    res = 1
    while x > 0:
        if x % 2 == 1:
            res = (res * c) % m
        x = x // 2
        c = (c * c) % m
    return res


def RSA_genkey(p,q):
    n = p*q
    phi = (p-1)*(q-1)
    e = 65537
    # while e < phi:
    #     if gcd(e,phi) == 1:
    #         break
    #     else:
    #         e += 1
    d = modinv(e,phi)
    public_key = (n,e)
    private_key = (n,d)
    return [public_key,private_key]

def fastmodexp(a, n, p):
    res = 1
    while n > 0:
        if n % 2 == 1:
            res = (res * a) % p
        n = n // 2
        a = (a * a) % p
    return res

# Python code for RSA using two prime number p and q

def RSA_genkey(p,q):
    n = p*q
    phi = (p-1)*(q-1)
    e = 65537
    # while e < phi:
    #     if gcd(e,phi) == 1:
    #         break
    #     else:
    #         e += 1
    d = modinv(e,phi)
    public_key = (n,e)
    private_key = (n,d)
    return [public_key,private_key]

def RSA_sign(digest,private_key):
    n = private_key[0]
    d = private_key[1]
    signature = fastmodexp(digest,d,n)
    return signature

def RSA_encrypt(key,public_key):
    n = public_key[0]
    e = public_key[1]
    cipher = fastmodexp(key,e,n)
    return cipher

def RSA_decrypt(ekey,private_key):
    n = private_key[0]
    d = private_key[1]
    key = fastmodexp(ekey,d,n)
    return key

def RSA_verify(signature,public_key):
    n = public_key[0]
    e = public_key[1]
    decrypted_sign = fastmodexp(signature,e,n)
    return decrypted_sign==signature

# Python code for ECC 

import matplotlib.pyplot as plt

def modinv(a, m):
    m0, x0, x1 = m, 0, 1
    while a > 1:
        q = a // m
        m, a = a % m, m
        x0, x1 = x1 - q * x0, x0
    return x1 + m0 if x1 < 0 else x1

def ECC_valid(a,b,p):
    return (4*(a**3)+27*(b**2))%p != 0

def ECC_genpts(a,b,p):
    pts = []
    for x in range(p):
        for y in range(p):
            if (y**2)%p == (x**3+a*x+b)%p:
                pts.append((x,y))
    pts.append(float('Inf'))
    return pts

def ECC_plotPxy(a,b,p):
    pts = ECC_genpts(a,b,p)
    pts = pts[:-1]
    x = []
    y = []
    for pt in pts:
        x.append(pt[0])
        y.append(pt[1])
    plt.scatter(x,y)
    plt.title("y^2 = x^3 + {}x + {} mod {}".format(a,b,p))
    plt.xlabel("x")
    plt.ylabel("y")
    plt.grid(True)
    plt.show()

def ECC_isPinE(P,a,b,p):
    return ((P[1]**2)%p) == ((P[0]**3+a*P[0]+b)%p)

def ECC_addPQ(P,Q,a,b,p):
    if P == float('Inf'):
        return Q
    elif Q == float('Inf'):
        return P
    elif P[0] == Q[0] and P[1] == -Q[1]:
        return float('Inf')
    elif P[0] == Q[0] and P[1] != Q[1]:
        return float('Inf')
    elif P[0] == Q[0] and P[1] == Q[1]:
        m = ((3*(P[0]**2)+a)%p)*modinv(2*P[1],p)
    else:
        m = ((P[1]-Q[1])%p)*modinv((P[0]-Q[0])%p,p)
    x = (m**2-P[0]-Q[0])%p
    y = (m*(P[0]-x)-P[1])%p
    return (x,y)
    
def ECC_orderP(P, a, b, p):
    i = 1
    if ECC_isPinE(P, a, b, p):
        P1 = P
        while P1 != float('Inf'):
            print(i, P1)
            P1 = ECC_addPQ(P, P1, a, b, p)
            # if P1 is None:
            #     break
            # if not ECC_isPinE(P1, a, b, p):
            #     break
            i += 1
        return i

def ECC_nP(n,P,a,b,p):
    if n == 0:
        return float('Inf')
    elif n == 1:
        return P
    else:
        return ECC_addPQ(P,ECC_nP(n-1,P,a,b,p),a,b,p)
    
def ECC_2nP(n,P,a,b,p):
    # This function returns the point 2^n*P, so if n = 3, it returns 8P
    if n == 0:
        return float('Inf')
    elif n == 1:
        return ECC_addPQ(P,P,a,b,p)
    else:
        i = 2**n
        while i > 1:
            P = ECC_addPQ(P,P,a,b,p)
            i -= 1
        return P

In [26]:
# Python code for ECC 

import matplotlib.pyplot as plt

def modinv(a, m):
    m0, x0, x1 = m, 0, 1
    while a > 1:
        q = a // m
        m, a = a % m, m
        x0, x1 = x1 - q * x0, x0
    return x1 + m0 if x1 < 0 else x1

def ECC_valid(a,b,p):
    return (4*(a**3)+27*(b**2))%p != 0

def ECC_genpts(a,b,p):
    pts = []
    for x in range(p):
        for y in range(p):
            if (y**2)%p == (x**3+a*x+b)%p:
                pts.append((x,y))
    pts.append(float('Inf'))
    return pts

def ECC_plotPxy(a,b,p):
    pts = ECC_genpts(a,b,p)
    pts = pts[:-1]
    x = []
    y = []
    for pt in pts:
        x.append(pt[0])
        y.append(pt[1])
    plt.scatter(x,y)
    plt.title("y^2 = x^3 + {}x + {} mod {}".format(a,b,p))
    plt.xlabel("x")
    plt.ylabel("y")
    plt.grid(True)
    plt.show()

def ECC_isPinE(P,a,b,p):
    return ((P[1]**2)%p) == ((P[0]**3+a*P[0]+b)%p)

def ECC_addPQ(P,Q,a,b,p):
    if P == float('Inf'):
        return Q
    elif Q == float('Inf'):
        return P
    elif P[0] == Q[0] and P[1] == -Q[1]:
        return float('Inf')
    elif P[0] == Q[0] and P[1] != Q[1]:
        return float('Inf')
    elif P[0] == Q[0] and P[1] == Q[1]:
        m = ((3*(P[0]**2)+a)%p)*modinv(2*P[1],p)
    else:
        m = ((P[1]-Q[1])%p)*modinv((P[0]-Q[0])%p,p)
    x = (m**2-P[0]-Q[0])%p
    y = (m*(P[0]-x)-P[1])%p
    return (x,y)
    
def ECC_orderP(P, a, b, p):
    i = 1
    if ECC_isPinE(P, a, b, p):
        P1 = P
        while P1 != float('Inf'):
            print(i, P1)
            P1 = ECC_addPQ(P, P1, a, b, p)
            # if P1 is None:
            #     break
            # if not ECC_isPinE(P1, a, b, p):
            #     break
            i += 1
        return i

def ECC_nP(n,P,a,b,p):
    if n == 0:
        return float('Inf')
    elif n == 1:
        return P
    else:
        return ECC_addPQ(P,ECC_nP(n-1,P,a,b,p),a,b,p)
    
def ECC_2nP(n,P,a,b,p):
    # This function returns the point 2^n*P, so if n = 3, it returns 8P
    if n == 0:
        return float('Inf')
    elif n == 1:
        return ECC_addPQ(P,P,a,b,p)
    else:
        i = 2**n
        while i > 1:
            P = ECC_addPQ(P,P,a,b,p)
            i -= 1
        return P